<a href="https://colab.research.google.com/github/Sazim2019331087/ML_Lab_Final/blob/main/Machine_Learning_Lab_(Masked_Face_Recognition).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installing Necessary Libraies**

In [1]:
pip install deepface tensorflow keras numpy opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.8 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=92a829375c33ac858c579da00ceb6c33709e613fe4faae3a0e939b088fe4897c
  Stored in directory: /root/.cache/pip/wheels/19/39/2f/2d3cadc408a8804103f1c34ddd4b9f6a93497b11fa96fe738e
Successfully built fire


**Generate and save encodings**

In [2]:
from deepface import DeepFace
import os
import csv
import json  # For encoding serialization

# Path to the folder containing unmasked images
resources_folder = "image without mask"
csv_file = "face_encodings.csv"

def generate_and_save_encodings():
    if not os.path.exists(resources_folder):
        print(f"The folder '{resources_folder}' does not exist.")
        return

    with open(csv_file, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["name", "encoding"])  # Write the CSV header

        # Loop through each image in the folder
        for filename in os.listdir(resources_folder):
            if filename.lower().endswith((".png", ".jpg", ".jpeg")):
                # Get the full path of the image
                image_path = os.path.join(resources_folder, filename)

                # Extract the name from the filename (without extension)
                name = os.path.splitext(filename)[0]

                try:
                    # Get face embedding using DeepFace
                    embedding = DeepFace.represent(image_path, model_name="Facenet")[0]["embedding"]

                    # Convert embedding (numpy array) to JSON-friendly format
                    embedding_serialized = json.dumps(embedding)

                    # Save the name and serialized embedding to the CSV
                    writer.writerow([name, embedding_serialized])
                    print(f"Processed {filename}")
                except Exception as e:
                    print(f"Error processing {filename}: {e}")

    print("Face encodings have been saved to", csv_file)

# Call the function
generate_and_save_encodings()


24-12-14 14:09:37 - Directory /root/.deepface has been created
24-12-14 14:09:37 - Directory /root/.deepface/weights has been created
24-12-14 14:09:40 - facenet_weights.h5 will be downloaded...


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/facenet_weights.h5
To: /root/.deepface/weights/facenet_weights.h5
100%|██████████| 92.2M/92.2M [00:00<00:00, 232MB/s]


Processed shamima.jpg
Processed zawad.jpg
Processed shafat.jpg
Processed razib.jpg
Processed urmi.jpg
Processed bijoy.jpg
Processed mrittika.jpg
Processed barak_obama.jpg
Processed billah.jpg
Processed kabir.jpg
Processed sazim.jpg
Processed ramzan_kadyrov.jpeg
Processed farhan.jpg
Processed masud.jpg
Processed ruhel.jpg
Processed afrina.jpg
Face encodings have been saved to face_encodings.csv


**Record Encodings**

In [6]:
import numpy as np

def load_encodings():
    encodings = {}
    with open(csv_file, mode="r") as file:
        reader = csv.reader(file)
        next(reader)  # Skip the header

        for row in reader:
            name = row[0]
            encoding = np.array(eval(row[1]))  # Convert string back to numpy array
            encodings[name] = encoding

    return encodings


**Recognizing audio**

In [4]:
from scipy.spatial.distance import cosine

def recognize_face(image_path, known_encodings, threshold=0.5):
    try:
        # Generate embedding for the input image
        embedding = DeepFace.represent(image_path, model_name="Facenet")[0]["embedding"]

        # Compare with known embeddings
        for name, known_embedding in known_encodings.items():
            similarity = 1 - cosine(known_embedding, embedding)
            if similarity > threshold:
                return f"Matched with: {name} (similarity: {similarity:.2f})"

        return "No match found"
    except Exception as e:
        return f"Error recognizing face: {e}"


**Testing the whole thing**

In [10]:
# Step 1: Generate and save encodings
#generate_and_save_encodings()

# Step 2: Load encodings from CSV
#known_encodings = load_encodings()

# Step 3: Recognize a face with a mask
masked_image_path = "/content/image with mask/kabir.jpg"  # Replace with the actual image path
result = recognize_face(masked_image_path, known_encodings)
print(result)


No match found
